In [1]:
import numpy as np
np.random.seed(0)
from keras.models import load_model
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

import pandas as pd

from sklearn.metrics import precision_recall_fscore_support,accuracy_score,recall_score,confusion_matrix,precision_score
from sklearn.svm import SVC

C:\Users\aliye\Anaconda3\envs\Audeep\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
TRAIN =True
timesteps=64
latent_dim=16
def extract_features(db):
    train_db=pd.read_csv("spectograms/MeanNormalized/compare18-heartbeat-0.32-0.16-128-"+db+".csv/train/compare18-heartbeat-0.32-0.16-128-"+db+".csv")
    val_db=pd.read_csv("spectograms/MeanNormalized/compare18-heartbeat-0.32-0.16-128-"+db+".csv/devel/compare18-heartbeat-0.32-0.16-128-"+db+".csv")
    class0=train_db[train_db["label_nominal"]==0]
    class1=train_db[train_db["label_nominal"]==1]
    class2=train_db[train_db["label_nominal"]==2]
    
 
    train_upsampled=pd.concat((class0,class0,class0,class1,class2,class2))
    train_upsampled=train_upsampled.reindex()
    train_Y=train_upsampled.label_nominal
    val_Y=val_db.label_nominal
    train_X=train_upsampled.iloc[:,4:]
    val_X=val_db.iloc[:,4:]
    

    
    train_X=train_X.values.flatten().reshape(len(train_X),timesteps,372) #502,64,372
    input_dim=train_X.shape[2]
    
    inputs = Input(shape=(timesteps, input_dim))
    encoded = LSTM(latent_dim)(inputs)

    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(input_dim, return_sequences=True)(decoded)

    sequence_autoencoder = Model(inputs, decoded)
    
    encoder = Model(inputs, encoded) #hidden representations 
    if TRAIN:


        sequence_autoencoder.compile(optimizer='adam', loss='mse')
        early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
        sequence_autoencoder.fit(train_X,train_X,batch_size=32,epochs=150,callbacks=[early_stop])
        sequence_autoencoder.save("models/upsample/autoencoder_2layer_"+str(timesteps)+"_"+db+"_"+str(latent_dim)+".model")

    else :
        sequence_autoencoder=load_model("models/upsample/autoencoder_2layer_"+str(timesteps)+"_"+db+"_"+str(latent_dim)+".model")
    features_train_X=encoder.predict(train_X)
    features_val_X=encoder.predict(val_X.values.reshape(len(val_X),timesteps,372))
    return features_train_X,features_val_X,train_Y,val_Y

In [3]:
train_75_X,val_75_X,train_75_Y,val_75_Y=extract_features("75")
train_60_X,val_60_X,train_60_Y,val_60_Y=extract_features("60")
train_45_X,val_45_X,train_45_Y,val_45_Y=extract_features("45")
train_30_X,val_30_X,train_30_Y,val_30_Y=extract_features("30")
# all Y's are same

Epoch 1/150
812/812 [==============================] - 8s 9ms/step - loss: 0.0985
Epoch 2/150
812/812 [==============================] - 6s 7ms/step - loss: 0.0804
Epoch 3/150
812/812 [==============================] - 6s 7ms/step - loss: 0.0791
Epoch 4/150
812/812 [==============================] - 6s 7ms/step - loss: 0.0785
Epoch 5/150
812/812 [==============================] - 6s 7ms/step - loss: 0.0776
Epoch 6/150
812/812 [==============================] - 6s 7ms/step - loss: 0.0770
Epoch 7/150
812/812 [==============================] - 6s 7ms/step - loss: 0.0764
Epoch 8/150
812/812 [==============================] - 6s 7ms/step - loss: 0.0781
Epoch 9/150
812/812 [==============================] - 6s 7ms/step - loss: 0.0800
Epoch 00009: early stopping
Epoch 1/150
812/812 [==============================] - 6s 8ms/step - loss: 0.1036
Epoch 2/150
812/812 [==============================] - 6s 7ms/step - loss: 0.0844
Epoch 3/150
812/812 [==============================] - 6s 8ms/step - l

In [4]:

train_X=np.concatenate((train_30_X,train_45_X,train_60_X,train_75_X),axis=1)
val_X=np.concatenate((val_30_X,val_45_X,val_60_X,val_75_X),axis=1)

In [72]:

model = Sequential()
model.add(Dense(64, input_dim=4*latent_dim,activation="sigmoid"))
model.add(Dense(64,activation="sigmoid"))

model.add(Dense(3))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
train_Y=to_categorical(train_75_Y)
val_Y=to_categorical(val_75_Y)

In [73]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
model.fit(train_X,train_Y,batch_size=8,epochs=50,validation_data=(val_X, val_Y),callbacks=[early_stop])

predictions=np.argmax(model.predict(val_X),axis=1)
print("Recall: ",recall_score(val_30_Y,predictions,average="weighted"))
print("Accuracy: ",accuracy_score(val_30_Y,predictions))
confusion_matrix(val_45_Y,predictions)

Train on 812 samples, validate on 180 samples
Epoch 1/50
812/812 [==============================] - 2s 2ms/step - loss: 1.1138 - acc: 0.3645 - val_loss: 1.0697 - val_acc: 0.2778
Epoch 2/50
812/812 [==============================] - 0s 505us/step - loss: 1.1102 - acc: 0.3079 - val_loss: 1.0959 - val_acc: 0.4056
Epoch 3/50
812/812 [==============================] - 0s 508us/step - loss: 1.1073 - acc: 0.3436 - val_loss: 1.1044 - val_acc: 0.2778
Epoch 4/50
812/812 [==============================] - 0s 515us/step - loss: 1.1034 - acc: 0.3485 - val_loss: 1.1120 - val_acc: 0.2778
Epoch 5/50
812/812 [==============================] - 0s 512us/step - loss: 1.1043 - acc: 0.3399 - val_loss: 1.0884 - val_acc: 0.4333
Epoch 6/50
812/812 [==============================] - 0s 505us/step - loss: 1.1000 - acc: 0.3436 - val_loss: 1.1540 - val_acc: 0.2778
Epoch 00006: early stopping
Recall:  0.2777777777777778
Accuracy:  0.2777777777777778


array([[ 0,  0, 32],
       [ 0,  0, 98],
       [ 0,  0, 50]], dtype=int64)

In [33]:

model = Sequential()
model.add(Dense(64, input_dim=latent_dim,activation="relu"))
model.add(Dense(64, input_dim=latent_dim,activation="relu"))

model.add(Dense(3))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
train_Y=to_categorical(train_75_Y)
val_Y=to_categorical(val_75_Y)
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
model.fit(train_75_X,train_Y,batch_size=8,epochs=50,validation_data=(val_75_X, val_Y),callbacks=[early_stop])

predictions=np.argmax(model.predict(val_75_X),axis=1)
print("Recall: ",recall_score(val_30_Y,predictions,average="weighted"))
print("Accuracy: ",accuracy_score(val_30_Y,predictions))
confusion_matrix(val_45_Y,predictions)

Train on 812 samples, validate on 180 samples
Epoch 1/50
812/812 [==============================] - 1s 2ms/step - loss: 7.6755 - acc: 0.3485 - val_loss: 8.3405 - val_acc: 0.5222
Epoch 2/50
812/812 [==============================] - 0s 499us/step - loss: 7.2452 - acc: 0.3473 - val_loss: 8.3405 - val_acc: 0.5222
Epoch 3/50
812/812 [==============================] - 0s 493us/step - loss: 7.2452 - acc: 0.3473 - val_loss: 8.3405 - val_acc: 0.5222
Epoch 4/50
812/812 [==============================] - 0s 500us/step - loss: 7.2452 - acc: 0.3473 - val_loss: 8.3405 - val_acc: 0.5222
Epoch 00004: early stopping
Recall:  0.5222222222222223
Accuracy:  0.5222222222222223


array([[ 0, 30,  2],
       [ 0, 94,  4],
       [ 0, 50,  0]], dtype=int64)

In [25]:
train_75_X.shape

(812, 16)

In [76]:
svm_model=SVC(C=10,kernel="linear")
svm_model.fit(train_X,train_30_Y)
predictions=svm_model.predict(val_X)
print("Recall: ",recall_score(val_30_Y,predictions,average="weighted"))
print("Accuracy: ",accuracy_score(val_30_Y,predictions))
confusion_matrix(val_45_Y,predictions)

Recall:  0.4166666666666667
Accuracy:  0.4166666666666667


array([[13,  8, 11],
       [21, 42, 35],
       [10, 20, 20]], dtype=int64)